### <u>Cross-Clustered Models</u>
The varying effects with grouping by province worked relatively well, but some of the data, e.g. in the CARB and NWCS provinces show some complex structure that might benefit from more multiple grouping. Moreover, both model comparison methods, the WAIC and LOOCV, flagged max(blue) based clustering as potentially useful in combination with the province-based clustering. In this post I will further the analysis with two cross-classified models. Cross-classified models feature more than one groupings in paralle. Here I will use two pairs of groupings to develop two different models. The first model combines the max(blue) grouping with the province grouping. For good measure, I'll implement a second model combining max(blue) and biome grouping. For both models and for both groupings, I will implement covarying effects, since these offer some improvement on the fit.

In [1]:
import pickle
# Importing libraries
import pymc3 as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
from seaborn import PairGrid, kdeplot, jointplot, FacetGrid
import theano.tensor as tt
import re
from matplotlib import cm
from cmocean import cm as cmo
from matplotlib.colors import rgb2hex
from sklearn.preprocessing import StandardScaler

In [2]:
# Setting graphics...
% matplotlib inline
pl.style.use('bmh')
rcParams['xtick.labelsize'] = 16
rcParams['ytick.labelsize'] = 16
rcParams['axes.titlesize'] = 18
rcParams['axes.formatter.limits'] = (-2, 3)
rcParams['axes.labelsize'] = 16
rcParams['font.size'] = 16
rcParams['figure.titlesize'] = 20
rcParams['axes.titlesize'] = 18

In [3]:
df = pd.read_pickle('./pickleJar/df_log.pkl')
df.head().T

,0,1,2,3,4
id,4045,4056,4057,5949,5950
datetime,1998-08-29 17:55:00,1998-09-02 15:52:00,1998-09-02 18:10:00,1997-10-12 23:38:00,1997-10-13 18:33:00
lat,60.587,60.674,60.604,59.118,58.792
lon,-146.409,-147.682,-147.205,-148.677,-148.493
etopo2_l,1.91381,1.14613,2.02938,2.15534,2.45637
oisst,13.04,11.74,11.74,9.71,9.84
sal_l,NaN,NaN,NaN,NaN,NaN
rrs411_l,-2.48181,-2.69759,-2.57346,-2.74698,-2.85684
rrs443_l,-2.39369,-2.57914,-2.48391,-2.66945,-2.77425
rrs489_l,-2.32961,-2.44558,-2.39021,-2.56902,-2.6778


In [4]:
blue_mapping = dict(enumerate(df['mxBlId'].astype('category').cat.categories))
biome_mapping = dict(enumerate(df['biome'].astype('category').cat.categories))
prov_mapping = dict(enumerate(df['provCode'].astype('category').cat.categories))

In [5]:
d_ = df[['mxBlId','provCat','mxBlCat', 'biomCat', 'MxBl-Gr',
         'oisst', 'etopo2_l', 'chl', 'chl_l']].dropna()
d_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4126 entries, 0 to 4458
Data columns (total 9 columns):
mxBlId      4126 non-null object
provCat     4126 non-null int8
mxBlCat     4126 non-null int8
biomCat     4126 non-null int8
MxBl-Gr     4126 non-null float64
oisst       4126 non-null float64
etopo2_l    4126 non-null float64
chl         4126 non-null float64
chl_l       4126 non-null float64
dtypes: float64(5), int8(3), object(1)
memory usage: 237.7+ KB


In [6]:
n_provinces = d_.provCat.unique().size
n_blues = d_.mxBlCat.unique().size
n_biomes = d_.biomCat.unique().size
provIdx = d_.provCat.values
blueIdx = d_.mxBlCat.values
biomeIdx = d_.biomCat.values
log_band_ratio = d_['MxBl-Gr'].values
log_depth = d_['etopo2_l'].values
sst = d_['oisst'].values
predicted = d_['chl_l'].values

In [7]:
standard_scaler = StandardScaler()
log_band_ratio_s = standard_scaler.fit_transform(log_band_ratio.reshape(-1, 1))

In [ ]:
with pm.Model() as mdl_provblue_cov_nc:
    # hyperprior
    sd_dist = pm.HalfCauchy.dist(beta=2)
    # cov hyperprior
    pchol_prov = pm.LKJCholeskyCov('pchol_prov', eta=4, n=2, sd_dist=sd_dist)
    pchol_blue = pm.LKJCholeskyCov('pchol_blue' ,eta=4, n=2, sd_dist=sd_dist)
    
    # priors
    # ----------
    # convert triangular cholesky factor matrix to 2D array
    chol_prov = pm.expand_packed_triangular(2, pchol_prov, lower=True)
    chol_blue = pm.expand_packed_triangular(2, pchol_blue, lower=True)
    
    ab_main = pm.Normal('ab_main', 0, 10, shape=2)
    ab_blue_prior = pm.Normal('ab_blue_offset', 0, 10, shape=(2, n_blues))
    ab_prov_prior = pm.Normal('ab_prov_offset', 0, 10, shape=(2, n_provinces))
    
    ab_blue = tt.dot(chol_blue, ab_blue_prior) 
    ab_prov = tt.dot(chol_prov, ab_prov_prior)
    
    #a_blue = pm.Deterministic('a_blue', ab_blue[0])
    #b_blue = pm.Deterministic('b_blue', ab_blue[1])
    #a_prov = pm.Deterministic('a_prov', ab_prov[0])
    #b_prov = pm.Deterministic('b_prov', ab_prov[1])
    #sigma_ab_prov = pm.Deterministic('sigma_ab_prov', tt.sqrt(tt.diag(ab_prov)))
    #sigma_ab_blue = pm.Deterministic('sigma_ab_blue', tt.sqrt(tt.diag(ab_blue)))
    #corr_prov = tt.diag(sigma_ab_prov**-1).dot(ab_prov.dot(tt.diag(sigma_ab_prov**-1)))
    #corr_blue = tt.diag(sigma_ab_blue**-1).dot(ab_blue.dot(tt.diag(sigma_ab_blue**-1)))
    #rho_prov = pm.Deterministic('rho_prov', corr_prov[np.triu_indices(2, k=1)])
   # rho_blue = pm.Deterministic('rho_blue', corr_blue[np.triu_indices(2, k=1)])
    
    # forward model & model error
    intercepts = ab_main[0] + ab_prov[0, provIdx] + ab_blue[0, blueIdx]
    slopes = ab_main[1] + ab_prov[1, provIdx] + ab_blue[1, blueIdx]
    mu_ = intercepts + slopes * log_band_ratio
    eps = pm.HalfCauchy('eps', beta=2)
    #likelihood
    log_chl = pm.Normal('log_chl', mu=mu_, sd=eps, observed=predicted)
    trace_prov_cov_nc = pm.sample(2000, tune=1000, nuts_kwargs=dict(target_accept=0.9))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps_log__, ab_prov_offset, ab_blue_offset, ab_main, pchol_blue_cholesky_cov_packed__, pchol_prov_cholesky_cov_packed__]
  7%|▋         | 214/3000 [01:30<19:35,  2.37it/s]